In [1]:
%load_ext autoreload
%autoreload 2

# MLflow Classification Recipe Notebook

This notebook runs the MLflow Classification Recipe on Databricks and inspects its results. For more information about the MLflow Classification Recipe, including usage examples, see the [Classification Recipe overview documentation](https://mlflow.org/docs/latest/recipes.html#classification-recipe) the [Classification Recipe API documentation](https://mlflow.org/docs/latest/python_api/mlflow.recipes.html#module-mlflow.recipes.classification.v1.recipe).

In [2]:
from mlflow.recipes import Recipe

r = Recipe(profile="local")
r.clean()

2023/04/11 16:12:52 INFO mlflow.recipes.recipe: Creating MLflow Recipe 'mlflow-recipes-titanic' with profile: 'local'


In [3]:
# Display the DAGs (Directed Acyclic Graph)
# r.inspect()

In [4]:
r.run("ingest")

2023/04/11 16:12:53 INFO mlflow.recipes.step: Running step ingest...


In [5]:
r.run("split")

2023/04/11 16:12:53 INFO mlflow.recipes.utils.execution: ingest: No changes. Skipping.


Run MLFlow Recipe step: split
2023/04/11 16:12:54 INFO mlflow.recipes.step: Running step split...


In [6]:
training_data = r.get_artifact("training_data")
training_data.describe()

,Survived,Pclass,SibSp,Parch,Fare,Age
count,712.000000,712.000000,712.000000,712.000000,712.000000,570.000000
mean,0.386236,2.318820,0.544944,0.376404,31.362476,29.475877
std,0.487228,0.830694,1.142801,0.782297,47.619466,14.689461
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.670000
25%,0.000000,2.000000,0.000000,0.000000,7.895800,20.000000
50%,0.000000,3.000000,0.000000,0.000000,14.454200,28.000000
75%,1.000000,3.000000,1.000000,0.000000,30.695800,38.000000
max,1.000000,3.000000,8.000000,6.000000,512.329200,80.000000


In [7]:
r.run("transform")

2023/04/11 16:12:55 INFO mlflow.recipes.utils.execution: ingest, split: No changes. Skipping.


Run MLFlow Recipe step: transform
2023/04/11 16:12:56 INFO mlflow.recipes.step: Running step transform...


Name,Type
Survived,int64
Pclass,int64
Sex,object
SibSp,int64
Parch,int64
Fare,float64
Age,float64
Embarked,object
Name,Type
onehot__Pclass_1,float64


In [8]:
# r.run("train")

In [9]:
# trained_model = r.get_artifact("model")
# print(trained_model)

In [10]:
# r.run("evaluate")

In [11]:
# r.run("register")